In [ ]:
import pandas as pd
import re

In [ ]:
# 1. 파일 경로
input_file = "data/raw/Y06_안전비상벨_TM.csv"
output_file = "data/processed/Y06_안전비상벨_TM_전처리.csv"

In [ ]:
# 2. 데이터 불러오기 (인코딩 이중 시도)
try:
    df = pd.read_csv(input_file, encoding="utf-8-sig")
except UnicodeDecodeError:
    df = pd.read_csv(input_file, encoding="cp949")

print(f"불러온 데이터 shape: {df.shape}")
print(f"컬럼 목록: {df.columns.tolist()}")
